# import + functions

In [4]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import os
from moviepy.editor import *


def get_csv_speed_align_time(Tracking_sleep_data,align_path,save):
    Task_body_centre=get_dlc_data(Tracking_sleep_data,'body_centre',False,0.9995)
    Task_body_centre_2=get_dlc_data(Tracking_sleep_data,'body_centre',True,0.9995)
    distances=get_distances(Task_body_centre_2)
    align = open(align_path, 'r')
    reader = csv.reader(align)
    headers = next(reader, None)
    column = {}
    for h in headers:
        column[h] = []
    for row in reader:
        for h, v in zip(headers, row):
            column[h].append(v)
    Camera_EPhys_aligned=np.array(column['Camera_time_Ephys_Aligned'])    
    align.close()
    fig, axs = plt.subplots(4, 1,figsize=(6, 15))
    axs[0].plot(Task_body_centre['x'].values,'o',color = 'blue',markersize=2)
    axs[0].plot(Task_body_centre_2['interped_x'].values,'o',color = 'red',markersize=1)
    axs[0].set_title('X')
    axs[1].plot(Task_body_centre['y'].values,'o',color = 'blue',markersize=2)
    axs[1].plot(Task_body_centre_2['interped_y'].values,'o',color = 'red',markersize=1)
    axs[1].set_title('Y')
    axs[2].plot(Task_body_centre['x'].values,Task_body_centre['y'].values,'o',color = 'blue',markersize=2)
    axs[2].plot(Task_body_centre_2['interped_x'].values,Task_body_centre_2['interped_y'].values,'o',color = 'red',markersize=1)
    axs[3].plot(distances,color = 'blue',markersize=2)
    df = pd.DataFrame({"v" : distances, "Ephys" : Camera_EPhys_aligned})
    df.to_csv(save, index=False)    
    return Camera_EPhys_aligned , distances

def get_dlc_data(Tracking_sleep_data,name,interp,val):
    # Load in all '.h5' files for a given folder:
    TFiles_unsort = list_files(Tracking_sleep_data, 'h5')
    for file in TFiles_unsort:
        if 'ABOVE' in file:
            h5_read=pd.read_hdf(Tracking_sleep_data+file)
    # Access the head center      
    scorer =  h5_read.columns.tolist()[0][0]
    Task_body_centre =  h5_read[scorer][name]
    print(Task_body_centre)
    if interp:
        Task_body_centre=clean_and_interpolate(Task_body_centre,val)
        print(Task_body_centre)
        

    return  Task_body_centre

def get_distances(Task_body_centre_2):
    distance=[]
    for i, x in enumerate(Task_body_centre_2['interped_x'].values):
        if i== 0:
            x_vector_old = Task_body_centre_2['interped_y'].values[i]
            y_vector_old = Task_body_centre_2['interped_x'].values[i]
        else:
            x_vector_new = Task_body_centre_2['interped_y'].values[i]
            y_vector_new = Task_body_centre_2['interped_x'].values[i]
        
            distance_x = (x_vector_new -x_vector_old)**2 
            distance_y = (y_vector_new -y_vector_old)**2 
        
            distance.append(np.sqrt(distance_x + distance_y))
            y_vector_old = y_vector_new
            x_vector_old = x_vector_new
    distance.append(np.sqrt(distance_x + distance_y))   # Repeat the last row to have the same size vector     
    return distance

def list_files(directory, extension):
    return (f for f in os.listdir(directory) if f.endswith('.' + extension))

def clean_and_interpolate(clock_head_centre,threshold):

    bad_confidence_inds = np.where(clock_head_centre.likelihood.values<threshold)[0]
    newx = clock_head_centre.x.values
    newx[bad_confidence_inds] = 0
    newy = clock_head_centre.y.values
    newy[bad_confidence_inds] = 0

    start_value_cleanup(newx)
    interped_x = interp_0_coords(newx)

    start_value_cleanup(newy)
    interped_y = interp_0_coords(newy)
    
    clock_head_centre['interped_x'] = interped_x
    clock_head_centre['interped_y'] = interped_y
    
    return clock_head_centre

def start_value_cleanup(coords):
    # This is for when the starting value of the coords == 0; interpolation will not work on these coords until the first 0 
    #is changed. The 0 value is changed to the first non-zero value in the coords lists
    for index, value in enumerate(coords):
        working = 0
        if value > 0:
            start_value = value
            start_index = index
            working = 1
            break
    if working == 1:
        for x in range(start_index):
            coords[x] = start_value
            
def interp_0_coords(coords_list):
    #coords_list is one if the outputs of the get_x_y_data = a list of co-ordinate points
    for index, value in enumerate(coords_list):
        if value == 0:
            if coords_list[index-1] > 0:
                value_before = coords_list[index-1]
                interp_start_index = index-1
                

        if index < len(coords_list)-1:
            if value ==0:
                if coords_list[index+1] > 0:
                    interp_end_index = index+1
                    value_after = coords_list[index+1]
                   

                    #now code to interpolate over the values
                    try:
                        interp_diff_index = interp_end_index - interp_start_index
                    except UnboundLocalError:
#                         print('the first value in list is 0, use the function start_value_cleanup to fix')
                        break
                 

                    new_values = np.linspace(value_before, value_after, interp_diff_index)
                    #print(new_values)

                    interp_index = interp_start_index+1
                    for x in range(interp_diff_index):
                        
                        coords_list[interp_index] = new_values[x]
                        interp_index +=1
        if index == len(coords_list)-1:
            if value ==0:
                for x in range(30):
                    coords_list[index-x] = coords_list[index-30]
                    #print('')
    print('function exiting')
    return(coords_list)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]
                  

# set paths 

In [24]:
path = r'Z:\projects\sequence_squad\revision_data\organised_data\animals\\'

animal = "seq006_implant1"

# main loop

In [ ]:
path_ = path+animal
for recording in os.listdir(path_)[4::]:
    if not 'Store' in recording: # ignore ds store thing
        print(recording)
        current_path = os.path.join(path_,recording) + '\\'
        output_path = os.path.join(path_,recording,'post_process_ppseq') + '\\' 
        
        # presleep        
        video_path = os.path.join(current_path, r"video/videos//")
        Tracking_sleep_data = os.path.join(current_path, r"video/tracking//")
        align_path = os.path.join(current_path, r"behav_sync/1_pre_sleep//") + "Presleep_Ephys_Camera_sync.csv"
        
        try:
            Camera_EPhys_aligned, distances =get_csv_speed_align_time(Tracking_sleep_data,align_path,output_path + 'velocity_mice_1_presleep.csv')
        except:
            print('no presleep?')
    
        # post sleep
        video_path = os.path.join(current_path, r"video/videos//")
        Tracking_sleep_data = os.path.join(current_path, r"video/tracking//")
        align_path = os.path.join(current_path, r"behav_sync/3_post_sleep//") + "Postsleep_Ephys_Camera_sync.csv"

        try:
            Camera_EPhys_aligned, distances =get_csv_speed_align_time(Tracking_sleep_data,align_path,output_path + 'velocity_mice_3_post_sleep.csv')       
        except:
            print('no postsleep')
        
        
        


recording3_18-11-2024
